In [1]:
import tkinter as tk
from tkinter import filedialog
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill
import pandas as pd
import re  # Import regular expression module

# Function to check for inconsistencies and highlight them
def check_and_highlight_inconsistencies(file_path):
    df = pd.read_excel(file_path)
    wb = load_workbook(file_path)
    ws = wb.active

    # Create a new sheet for logging errors and matrix representation
    error_sheet = wb.create_sheet(title="Error Log")
    error_sheet.append(["Column Name", "Row Number", "Error Type", "Cell Value"])
    matrix_sheet = wb.create_sheet(title="Error Matrix")

    # Initialize an empty matrix to mark errors (same shape as input data)
    error_matrix = [[0] * len(df.columns) for _ in range(len(df))]

    # Define the fills for different inconsistency types
    missing_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')  # Red for missing
    invalid_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')  # Yellow for non-numeric
    negative_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')  # Green for negatives
    pattern_fill = PatternFill(start_color='FF00FF', end_color='FF00FF', fill_type='solid')  # Magenta for misspelling

    # Column-specific rules (you can update this)
    column_rules = {
        'Payload ID (rdrc_name)': ['missing', 'non-numeric'],
        'Payload': ['pattern'],
        'Radio Frequency Identification (RFID)': ['missing', 'non-numeric'],
        'Age at euthanasia': ['missing', 'negative'],
        'Sample name': ['pattern'],
        'Type': ['pattern'],
        'Section': ['pattern'],
        'Strain': ['pattern'],
        'Genotype': ['pattern'],
        'Sex': ['pattern'],
        # Add more columns with their rules here
    }

    # Function to check inconsistencies based on rules
    def check_column_inconsistencies(column_name, column_data):
        rules = column_rules.get(column_name, [])
        
        # Check for missing values
        if 'missing' in rules:
            for i, val in column_data.items():
                if pd.isnull(val):
                    ws.cell(row=i+2, column=col_idx).fill = missing_fill
                    error_sheet.append([column_name, i+2, 'Missing value', None])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix
        
        # Check for non-numeric values
        if 'non-numeric' in rules and pd.api.types.is_numeric_dtype(column_data):
            for i, val in column_data.items():
                if pd.isnull(pd.to_numeric(val, errors='coerce')):
                    ws.cell(row=i+2, column=col_idx).fill = invalid_fill
                    error_sheet.append([column_name, i+2, 'Non-numeric value', val])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix
        
        # Check for negative values
        if 'negative' in rules and pd.api.types.is_numeric_dtype(column_data):
            for i, val in column_data.items():
                if val < 0:
                    ws.cell(row=i+2, column=col_idx).fill = negative_fill
                    error_sheet.append([column_name, i+2, 'Negative value', val])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix
        
        # Check for pattern in 'Sample name' (enhanced rule for misspelling or wrong format)
        if 'pattern' in rules and column_name == 'Sample name':
            pattern = re.compile(r'.*Heart$')
            for i, val in column_data.items():
                if isinstance(val, str) and not pattern.match(val):  # If string does not match the pattern
                    ws.cell(row=i+2, column=col_idx).fill = pattern_fill
                    error_sheet.append([column_name, i+2, 'Misspelling or incorrect pattern', val])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix
       
        # List of acceptable spellings for "Type"
        acceptable_spellings = ['Heart', 'Feces', 'Ileum', 'Jejunum', 'Stomach']

        # Check for pattern in 'Type' column (validate against acceptable spellings)
        if 'pattern' in rules and column_name == 'Type':
            for i, val in column_data.items():
                if isinstance(val, str) and val not in acceptable_spellings:
                    ws.cell(row=i+2, column=col_idx).fill = pattern_fill
                    error_sheet.append([column_name, i+2, 'Misspelling or unacceptable value', val])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix
                    
        # Check for specific acceptable value in 'Payload' column
        if 'pattern' in rules and column_name == 'Payload':
            acceptable_payload = 'RR-1'  # Define the acceptable value
            for i, val in column_data.items():
                if val != acceptable_payload:  # Check if the value does not match
                    ws.cell(row=i+2, column=col_idx).fill = pattern_fill
                    error_sheet.append([column_name, i+2, 'Unacceptable Payload value', val])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix

        # List of acceptable spellings for "Section"
        acceptable_sections = ['Pellet', 'Proximal', 'Distal', 'Partial', 'Fundus']
        
        # Check for pattern in 'Section' column (validate against acceptable spellings)
        if 'pattern' in rules and column_name == 'Section':
            for i, val in column_data.items():
                if isinstance(val, str) and val not in acceptable_sections:
                    ws.cell(row=i+2, column=col_idx).fill = pattern_fill
                    error_sheet.append([column_name, i+2, 'Unacceptable Section value', val])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix

        # Check for specific acceptable value in 'Strain' column
        if 'pattern' in rules and column_name == 'Strain':
            acceptable_strain = 'C57BL/6J'  # Define the acceptable value
            for i, val in column_data.items():
                if val != acceptable_strain:  # Check if the value does not match
                    ws.cell(row=i+2, column=col_idx).fill = pattern_fill
                    error_sheet.append([column_name, i+2, 'Unacceptable Strain value', val])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix
        
        # Check for specific acceptable value in 'Genotype' column
        if 'pattern' in rules and column_name == 'Genotype':
            acceptable_genotype = 'Wild Type'  # Define the acceptable value
            for i, val in column_data.items():
                if val != acceptable_genotype:  # Check if the value does not match
                    ws.cell(row=i+2, column=col_idx).fill = pattern_fill
                    error_sheet.append([column_name, i+2, 'Unacceptable Genotype value', val])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix
        
        # Check for specific acceptable value in 'Sex' column
        if 'pattern' in rules and column_name == 'Sex':
            acceptable_sex = 'Female'  # Define the acceptable value
            for i, val in column_data.items():
                if val != acceptable_sex:  # Check if the value does not match
                    ws.cell(row=i+2, column=col_idx).fill = pattern_fill
                    error_sheet.append([column_name, i+2, 'Unacceptable Sex value', val])
                    error_matrix[i][col_idx - 1] = 1  # Mark error in matrix

    # Loop through columns to check and highlight inconsistencies
    for col_idx, column in enumerate(df.columns, start=1):
        column_name = df.columns[col_idx - 1]
        check_column_inconsistencies(column_name, df[column_name])

    # Write the error matrix to the new sheet
    matrix_sheet.append([''] + list(df.columns))  # Header row with column names
    for i, row in enumerate(error_matrix):
        matrix_sheet.append([f'Row {i+2}'] + row)  # Row labels and matrix data

    # Save the updated file with highlights, error log, and matrix sheet
    output_file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", title="Save the updated Excel file")
    wb.save(output_file_path)
    print(f"File saved with highlighted inconsistencies, error log, and matrix: {output_file_path}")

# Function to handle the file selection and trigger processing
def select_file():
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx")])
    if file_path:
        check_and_highlight_inconsistencies(file_path)

# Create the GUI window
root = tk.Tk()
root.title("Excel Inconsistency Checker")

# Add a button to trigger file selection
button = tk.Button(root, text="Select Excel File", command=select_file)
button.pack(pady=20)

# Run the GUI loop
root.mainloop()


File saved with highlighted inconsistencies, error log, and matrix: /Users/tennisvish/Downloads/Fall_24_Walter_Project/SHmissingsextest1.xlsx
